In [1]:
pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer

# Wczytaj dane
df = pd.read_csv("final_dataset_1.csv")

# 1. Przygotowanie danych
df_model = df.drop(columns=['Unnamed: 0', 'Date', 'HomeTeam', 'AwayTeam', 'HTFormPtsStr', 'ATFormPtsStr'])

# 2. Zakoduj etykietę celu
le = LabelEncoder()
df_model['FTR'] = le.fit_transform(df_model['FTR'])  # H=0, D=1, A=2

# 3. Zamień W/D/L na liczby w kolumnach formy
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    df_model[col] = df_model[col].map(form_map)

# 4. Oddziel cechy i etykietę
X = df_model.drop(columns=['FTR'])
y = df_model['FTR']

# 5. Imputacja braków średnią
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 6. Podział na treningowy/testowy
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, stratify=y, random_state=42)

# 7. SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 8. Rozkład klas po SMOTE
print("Rozkład klas po SMOTE:")
print(pd.Series(y_resampled).value_counts())


Rozkład klas po SMOTE:
1    1635
0    1635
2    1635
Name: FTR, dtype: int64


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

# Wczytaj dane
df = pd.read_csv("final_dataset_1.csv")

# Usuń niepotrzebne kolumny
df_model = df.drop(columns=['Unnamed: 0', 'Date', 'HomeTeam', 'AwayTeam', 'HTFormPtsStr', 'ATFormPtsStr'])

# Zakoduj etykiety klas
le = LabelEncoder()
df_model['FTR'] = le.fit_transform(df_model['FTR'])  # H=0, D=1, A=2

# Zamień W/D/L na liczby
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    df_model[col] = df_model[col].map(form_map)

# Podziel dane
X = df_model.drop(columns=['FTR'])
y = df_model['FTR']

# Uzupełnij braki
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Podział na zbiór treningowy/testowy
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, stratify=y, random_state=42)

# Zastosuj SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Odtwórz etykiety klas
X_res_df = pd.DataFrame(X_resampled, columns=X.columns)
y_res_df = pd.Series(le.inverse_transform(y_resampled), name='FTR')

# Połącz i zapisz do CSV
balanced_df = pd.concat([X_res_df, y_res_df], axis=1)
balanced_df.to_csv("balanced_training_data.csv", index=False)
# Wyświetl rozkład klas po SMOTE
print("Rozkład klas po SMOTE:")
print(pd.Series(y_resampled).value_counts())




Rozkład klas po SMOTE:
1    1635
0    1635
2    1635
Name: FTR, dtype: int64


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

# Wczytaj dane
df = pd.read_csv("final_dataset_1.csv")

# Zakoduj kolumnę celu
le = LabelEncoder()
df['FTR_encoded'] = le.fit_transform(df['FTR'])  # H=0, D=1, A=2

# Zamień HM1-HM5, AM1-AM5 na liczby
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    df[col] = df[col].map(form_map)

# Przygotuj dane do SMOTE
X = df.drop(columns=['Unnamed: 0', 'Date', 'HomeTeam', 'AwayTeam', 'HTFormPtsStr', 'ATFormPtsStr', 'FTR', 'FTR_encoded'])
y = df['FTR_encoded']

# Uzupełnij NaN
X_imputed = SimpleImputer(strategy='mean').fit_transform(X)

# SMOTE
X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_imputed, y)

# Zamień na DataFrame
X_res_df = pd.DataFrame(X_resampled, columns=X.columns)
y_res_df = pd.Series(le.inverse_transform(y_resampled), name='FTR')

# Wylosuj metadane (Date, HomeTeam itd.) do uzupełnienia w takim samym formacie jak oryginalnie
meta = df[['Date', 'HomeTeam', 'AwayTeam', 'HTFormPtsStr', 'ATFormPtsStr']].sample(n=len(X_res_df), replace=True, random_state=42).reset_index(drop=True)

# Złącz dane
final_df = pd.concat([meta, X_res_df.reset_index(drop=True), y_res_df], axis=1)

# Zapis do pliku
final_df.to_csv("final_balanced_dataset.csv", index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

# Wczytaj dane
df = pd.read_csv("final_dataset_1.csv")

# Zapamiętaj oryginalną kolejność kolumn
original_columns = df.columns.tolist()

# Zakoduj etykietę celu
le = LabelEncoder()
df['FTR_encoded'] = le.fit_transform(df['FTR'])  # H=0, D=1, A=2

# Zamień 'W', 'D', 'L' na liczby w kolumnach formy meczowej
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    df[col] = df[col].map(form_map)

# Przygotuj dane do SMOTE
X_raw = df.drop(columns=['FTR', 'FTR_encoded'])
y = df['FTR_encoded']
X = X_raw.select_dtypes(include=['number'])

# Imputacja braków (średnia)
X_imputed = SimpleImputer(strategy='mean').fit_transform(X)

# SMOTE
X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_imputed, y)

# Konwersja z powrotem do DataFrame
X_res_df = pd.DataFrame(X_resampled, columns=X.columns)
y_res_labels = pd.Series(le.inverse_transform(y_resampled), name='FTR')

# Scal z powrotem z etykietą klasową
df_balanced = pd.concat([X_res_df.reset_index(drop=True), y_res_labels], axis=1)

# Wstaw kolumnę FTR w odpowiednie miejsce (tam, gdzie była w oryginale)
insert_position = original_columns.index('FTR')
df_balanced.insert(insert_position, 'FTR', df_balanced.pop('FTR'))

# Uporządkuj kolumny według oryginalnego porządku
final_df = df_balanced[original_columns]

# Zapisz do pliku
final_df.to_csv("final_balanced_dataset.csv", index=False)

# Wyświetl rozkład klas po SMOTE
print("Rozkład klas po SMOTE:")
print(pd.Series(y_resampled).value_counts())

KeyError: "['Date', 'HomeTeam', 'AwayTeam', 'HTFormPtsStr', 'ATFormPtsStr'] not in index"

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import numpy as np

# Wczytaj oryginalny plik
df = pd.read_csv("final_dataset_1.csv")

# Zapisz oryginalną kolejność kolumn
original_columns = df.columns.tolist()

# Zakoduj etykietę celu
le = LabelEncoder()
df['FTR_encoded'] = le.fit_transform(df['FTR'])  # H=0, D=1, A=2

# Zamień W/D/L na liczby w kolumnach formy meczowej
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    df[col] = df[col].map(form_map)

# Przygotuj dane do SMOTE (tylko kolumny numeryczne)
X = df.select_dtypes(include=['number']).drop(columns=['FTR_encoded'])
y = df['FTR_encoded']

# Imputacja braków
X_imputed = SimpleImputer(strategy='mean').fit_transform(X)

# SMOTE
X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_imputed, y)

# Odtwórz dane jako DataFrame
X_res_df = pd.DataFrame(X_resampled, columns=X.columns)
y_res_df = pd.Series(le.inverse_transform(y_resampled), name='FTR')

# Wylosuj kolumny tekstowe (Date, HomeTeam, AwayTeam, HTFormPtsStr, ATFormPtsStr)
meta_cols = ['Date', 'HomeTeam', 'AwayTeam', 'HTFormPtsStr', 'ATFormPtsStr']
meta_sample = df[meta_cols].sample(n=len(X_res_df), replace=True, random_state=42).reset_index(drop=True)

# Połącz całość
final_df = pd.concat([meta_sample.reset_index(drop=True), X_res_df.reset_index(drop=True), y_res_df], axis=1)

# Ustaw kolumny w oryginalnej kolejności
for col in original_columns:
    if col not in final_df.columns:
        final_df[col] = ""

final_df = final_df[original_columns]

# 🔁 Zamień floaty na int, jeśli nie ma miejsc po przecinku
def convert_column(col):
    if np.issubdtype(col.dtype, np.floating):
        if (col % 1 == 0).all():
            return col.astype(int)
    return col

final_df = final_df.apply(convert_column)

# 💾 Zapisz bez indexu i bez 'Unnamed: 0'
final_df.to_csv("final_balanced_dataset.csv", index=False)

print("✅ Plik zapisany jako final_balanced_dataset.csv z pełnym formatem.")


✅ Plik zapisany jako final_balanced_dataset.csv z pełnym formatem.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import numpy as np

# Wczytaj dane
df = pd.read_csv("final_dataset_1.csv")

# Usuń kolumnę indeksu jeśli istnieje
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# Zapamiętaj oryginalny układ kolumn
original_columns = df.columns.tolist()

# Zakoduj etykietę celu
le = LabelEncoder()
df['FTR_encoded'] = le.fit_transform(df['FTR'])

# Zamień W/D/L na liczby w formie meczowej
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    if col in df.columns:
        df[col] = df[col].map(form_map)

# Dane numeryczne do SMOTE
X = df.select_dtypes(include=['number']).drop(columns=['FTR_encoded'])
y = df['FTR_encoded']

# Imputacja braków (średnia)
X_imputed = SimpleImputer(strategy='mean').fit_transform(X)

# SMOTE – zbalansowanie danych
X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_imputed, y)

# Zamiana na DataFrame
X_res_df = pd.DataFrame(X_resampled, columns=X.columns)
y_res_df = pd.Series(le.inverse_transform(y_resampled), name='FTR')

# Wylosuj kolumny tekstowe do zachowania formatu
meta_cols = ['Date', 'HomeTeam', 'AwayTeam', 'HTFormPtsStr', 'ATFormPtsStr']
meta_sample = df[meta_cols].sample(n=len(X_res_df), replace=True, random_state=42).reset_index(drop=True)

# Połączenie danych
final_df = pd.concat([meta_sample, X_res_df.reset_index(drop=True), y_res_df], axis=1)

# Uzupełnij brakujące kolumny jeśli potrzeba
for col in original_columns:
    if col not in final_df.columns:
        final_df[col] = ""

# Przywróć oryginalną kolejność kolumn
final_df = final_df[original_columns]

# Zaokrąglenie bramek i innych kolumn logicznych do całkowitych
int_cols = ['FTHG', 'FTAG', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'MW',
            'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
            'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5']
for col in int_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].round().astype(int)

# Zapis do pliku CSV bez indeksu
final_df.to_csv("final_dataset_SMOTE_clean.csv", index=False)
print("✅ Zapisano: final_dataset_SMOTE_clean.csv")


✅ Zapisano: final_dataset_SMOTE_clean.csv


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import numpy as np

# Wczytaj dane
df = pd.read_csv("final_dataset_1.csv")

# Usuń Unnamed: 0 jeśli istnieje
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# Zapisz oryginalny porządek kolumn
original_columns = df.columns.tolist()

# Zakodowanie etykiety celu (FTR)
le = LabelEncoder()
df['FTR_encoded'] = le.fit_transform(df['FTR'])  # H=0, D=1, A=2

# Zamiana W/D/L na liczby
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    if col in df.columns:
        df[col] = df[col].map(form_map)

# Dane numeryczne do SMOTE
X = df.select_dtypes(include=['number']).drop(columns=['FTR_encoded'])
y = df['FTR_encoded']

# Sprawdzenie liczności klas
class_counts = y.value_counts()
max_class_count = class_counts.max()

# Przygotowanie strategii SMOTE tylko dla klas mniejszościowych
sampling_strategy = {label: max_class_count for label in class_counts.index if class_counts[label] < max_class_count}

# Jeśli jest coś do zrównania
if sampling_strategy:
    X_imputed = SimpleImputer(strategy='mean').fit_transform(X)
    smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_imputed, y)
else:
    # Jeśli już zbalansowane, nic nie robimy
    X_resampled, y_resampled = X.to_numpy(), y.to_numpy()

# Zamiana na DataFrame
X_res_df = pd.DataFrame(X_resampled, columns=X.columns)
y_res_df = pd.Series(le.inverse_transform(y_resampled), name='FTR')

# Losowe wstawienie kolumn tekstowych (z zachowaniem struktury)
meta_cols = ['Date', 'HomeTeam', 'AwayTeam', 'HTFormPtsStr', 'ATFormPtsStr']
meta_sample = df[meta_cols].sample(n=len(X_res_df), replace=True, random_state=42).reset_index(drop=True)

# Połączenie
final_df = pd.concat([meta_sample, X_res_df.reset_index(drop=True), y_res_df], axis=1)

# Dodaj brakujące kolumny jeśli trzeba
for col in original_columns:
    if col not in final_df.columns:
        final_df[col] = ""

# Przywrócenie oryginalnej kolejności kolumn
final_df = final_df[original_columns]

# Zaokrąglenie kolumn logicznych do int
int_cols = ['FTHG', 'FTAG', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'MW',
            'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
            'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5']
for col in int_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].round().astype(int)

# Zapisz gotowy plik
final_df.to_csv("final_dataset_SMOTE_balanced.csv", index=False)
print("✅ Zapisano: final_dataset_SMOTE_balanced.csv")


✅ Zapisano: final_dataset_SMOTE_balanced.csv
